In [1]:
import numpy as np
import pandas as pd
import time

In [5]:
ROWS = 3
COLS = 3
ACTIONS = ['up', 'down', 'left', 'right']
EPSILON = 0.9
ALPHA = 0.1
GAMMA = 0.9
MAX_EPISODES = 30
FRESH_TIME = 0.1
TRAP_POS = (1, 1)
GOAL_POS = (2, 2)

# 修改后的函数：预先生成所有状态并初始化 Q 表
def build_q_table(rows, cols, actions):
    states = [str((i, j)) for i in range(rows) for j in range(cols)]
    table = pd.DataFrame(
        np.zeros((len(states), len(actions))),
        columns=actions,
        index=states
    )
    return table

def choose_action(state, q_table):
    state_action = q_table.loc[state, :]
    if np.random.uniform() > EPSILON or (state_action == 0).all():
        action_name = np.random.choice(ACTIONS)
    else:
        action_name = state_action.idxmax()
    return action_name

def get_env_feedback(state_coords, action):
    row, col = state_coords
    if action == 'up':
        row = max(0, row - 1)
    elif action == 'down':
        row = min(ROWS - 1, row + 1)
    elif action == 'left':
        col = max(0, col - 1)
    elif action == 'right':
        col = min(COLS - 1, col + 1)
    next_coords = (row, col)
    if next_coords == GOAL_POS:
        reward = 1
        done = True
        return 'terminal', reward, done
    elif next_coords == TRAP_POS:
        reward = -1
        done = True
        return 'terminal', reward, done
    else:
        reward = 0
        done = False
        return str(next_coords), reward, done

def print_env(player_pos, episode, step):
    map_str = [[" . " for _ in range(COLS)] for _ in range(ROWS)]
    map_str[TRAP_POS[0]][TRAP_POS[1]] = " X "
    map_str[GOAL_POS[0]][GOAL_POS[1]] = " T "
    if player_pos != 'terminal':
        p_row, p_col = eval(player_pos)
        map_str[p_row][p_col] = " o "
    display = "\n".join(["".join(row) for row in map_str])
    print(f"\nEpisode: {episode + 1}, Step: {step}")
    print(display)
    time.sleep(FRESH_TIME)

def rl():
    q_table = build_q_table(ROWS, COLS, ACTIONS)
    for episode in range(MAX_EPISODES):
        state = str((0, 0))
        is_terminated = False
        step = 0
        show_anim = episode >= MAX_EPISODES - 5  # 最后5回合显示动画
        while not is_terminated:
            if show_anim:
                print_env(state, episode, step)
            action = choose_action(state, q_table)
            next_state, reward, is_terminated = get_env_feedback(eval(state), action)
            q_predict = q_table.loc[state, action]
            if next_state != 'terminal':
                q_target = reward + GAMMA * q_table.loc[next_state, :].max()
            else:
                q_target = reward
            q_table.loc[state, action] += ALPHA * (q_target - q_predict)
            state = next_state
            step += 1
    return q_table

if __name__ == "__main__":
    print("训练中...前25回合不显示,最后5回合展示成果")
    final_q = rl()
    print("\n训练结束!\n")
    print("查看起点 (0, 0) 的 Q 值表 (决策依据):")
    print(final_q.loc['(0, 0)'])

训练中...前25回合不显示,最后5回合展示成果

Episode: 26, Step: 0
 o  .  . 
 .  X  . 
 .  .  T 

Episode: 26, Step: 1
 .  o  . 
 .  X  . 
 .  .  T 

Episode: 26, Step: 2
 .  .  o 
 .  X  . 
 .  .  T 

Episode: 26, Step: 3
 .  .  . 
 .  X  o 
 .  .  T 

Episode: 27, Step: 0
 o  .  . 
 .  X  . 
 .  .  T 

Episode: 27, Step: 1
 .  o  . 
 .  X  . 
 .  .  T 

Episode: 27, Step: 2
 .  .  o 
 .  X  . 
 .  .  T 

Episode: 27, Step: 3
 .  .  . 
 .  X  o 
 .  .  T 

Episode: 28, Step: 0
 o  .  . 
 .  X  . 
 .  .  T 

Episode: 28, Step: 1
 .  o  . 
 .  X  . 
 .  .  T 

Episode: 29, Step: 0
 o  .  . 
 .  X  . 
 .  .  T 

Episode: 29, Step: 1
 .  o  . 
 .  X  . 
 .  .  T 

Episode: 29, Step: 2
 .  .  o 
 .  X  . 
 .  .  T 

Episode: 29, Step: 3
 .  .  . 
 .  X  o 
 .  .  T 

Episode: 30, Step: 0
 o  .  . 
 .  X  . 
 .  .  T 

Episode: 30, Step: 1
 o  .  . 
 .  X  . 
 .  .  T 

Episode: 30, Step: 2
 .  o  . 
 .  X  . 
 .  .  T 

Episode: 30, Step: 3
 .  .  o 
 .  X  . 
 .  .  T 

Episode: 30, Step: 4
 .  .  o 
 .  X  

In [8]:
# --- 7. 测试/演示函数 (只选最优解) ---
def test_agent(q_table):
    print("\n====================")
    print("开始最终成果演示 (100% 贪婪模式)")
    print("====================")
    
    state = str((0, 0)) # 回到起点，并确保是字符串
    step = 0
    is_terminated = False
    
    # 打印初始位置
    print_env(state, "TEST", step)
    
    while not is_terminated:
        # 核心区别：这里不再使用 choose_action (因为它有随机性)
        # 而是直接选 Q 值最大的动作
        
        # 2. 选分最高的动作 (idxmax)
        action = q_table.loc[state, :].idxmax()
        
        print(f"当前位置: {state}, 觉得最好的动作是: {action}")
        
        # 3. 执行动作
        next_state, reward, is_terminated = get_env_feedback(eval(state), action)
        
        # 4. 确保转成字符串防止类型错误
        state = str(next_state)
        step += 1
        
        print_env(state, "TEST", step)
        
        # 如果陷入死循环 (没训练好)，强制停止
        if step > 20:
            print("步数过多，可能陷入循环，测试强制停止。")
            break

    if reward == 1:
        print("🎉 演示成功！智能体避开了陷阱并拿到了宝藏。")
    elif reward == -1:
        print("☠️ 演示失败！智能体掉进了陷阱。")

# --- 修改主程序入口 ---
if __name__ == "__main__":
    print("1. 开始训练 (30回合)...")
    # 为了演示效果，训练时我们可以把动画关掉或者加快，这里直接用之前的逻辑
    final_q = rl()
    
    print("\n2. 训练结束，查看最终 Q 表片段:")
    print(final_q)
    
    # 运行测试
    import time
    time.sleep(1) # 停顿一下让我们准备好
    test_agent(final_q)

1. 开始训练 (30回合)...

Episode: 26, Step: 0
 o  .  . 
 .  X  . 
 .  .  T 

Episode: 26, Step: 1
 .  o  . 
 .  X  . 
 .  .  T 

Episode: 26, Step: 2
 .  .  o 
 .  X  . 
 .  .  T 

Episode: 26, Step: 3
 .  .  . 
 .  X  o 
 .  .  T 

Episode: 27, Step: 0
 o  .  . 
 .  X  . 
 .  .  T 

Episode: 27, Step: 1
 .  o  . 
 .  X  . 
 .  .  T 

Episode: 27, Step: 2
 .  .  o 
 .  X  . 
 .  .  T 

Episode: 27, Step: 3
 .  .  . 
 .  X  o 
 .  .  T 

Episode: 28, Step: 0
 o  .  . 
 .  X  . 
 .  .  T 

Episode: 28, Step: 1
 .  o  . 
 .  X  . 
 .  .  T 

Episode: 28, Step: 2
 .  .  o 
 .  X  . 
 .  .  T 

Episode: 28, Step: 3
 .  .  o 
 .  X  . 
 .  .  T 

Episode: 28, Step: 4
 .  .  . 
 .  X  o 
 .  .  T 

Episode: 29, Step: 0
 o  .  . 
 .  X  . 
 .  .  T 

Episode: 29, Step: 1
 .  o  . 
 .  X  . 
 .  .  T 

Episode: 29, Step: 2
 .  .  o 
 .  X  . 
 .  .  T 

Episode: 29, Step: 3
 .  .  . 
 .  X  o 
 .  .  T 

Episode: 30, Step: 0
 o  .  . 
 .  X  . 
 .  .  T 

Episode: 30, Step: 1
 o  .  . 
 .  X  . 
 .  

TypeError: can only concatenate str (not "int") to str